In [31]:
## Importing necessary Libraries 
import ctypes, inspect, os, graphlab
from graphlab import SFrame
import pandas as pd

In [84]:
# This code reads the data into a graphlab Sframe. The dataset used is an Office Products dataset containing 53,257 reviews   
# retrieved from: http://jmcauley.ucsd.edu/data/amazon/ citation: "Image-based recommendations on styles and 
# substitutes" J. McAuley, C. Targett, J. Shi, A. van den Hengel, SIGIR, 2015
Office_Products = graphlab.SFrame.read_json('Office_Products_5.json', orient='lines')

Finished parsing file C:\Users\Chauncey\Office_Products_5.json

Parsing completed. Parsed 100 lines in 1.31107 secs.

Read 50429 lines. Lines per second: 32575

Finished parsing file C:\Users\Chauncey\Office_Products_5.json

Parsing completed. Parsed 53258 lines in 1.57609 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [85]:
## Exploring the dataset
Office_Products.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName
B00000JBLH,"[3, 4]",5.0,"I bought my first HP12Cin about 1984 or so, and ...","09 3, 2004",A32T2H8150OJLU,ARH
B00000JBLH,"[7, 9]",5.0,WHY THIS BELATED REVIEW?I feel very obliged to ...,"12 15, 2007",A3MAFS04ZABRGO,"Let it Be ""Alan"""
B00000JBLH,"[3, 3]",2.0,I have an HP 48GX thathas been kicking for ...,"01 1, 2011",A1F1A0QQP2XVH5,Mark B
B00000JBLH,"[7, 8]",5.0,I've started doing morefinance stuff recently ...,"04 19, 2006",A49R5DBXXQDE5,R. D Johnson
B00000JBLH,"[0, 0]",5.0,For simple calculationsand discounted cash ...,"08 4, 2013",A2XRMQA6PJ5ZJ8,Roger J. Buffington
B00000JBLH,"[10, 12]",5.0,"While I don't have anMBA, it's hard to bel ...","01 23, 2002",A2JFOHC9W629IE,scott_from_dallas
B00000JBLH,"[3, 4]",5.0,I've had an HP 12C eversince they were first ...,"01 17, 2007",A38NELQT98S4H8,W. B. Halper
B00000JBLH,"[0, 0]",5.0,Bought this for my bossbecause he lost his. He ...,"11 14, 2013",AA8M6331NI1EN,ZombieMom
B00000JBLU,"[3, 3]",5.0,"This is a well-designed,simple calculator that ...","12 7, 2010",A25C2M3QF9G7OQ,Comdet
B00000JBLU,"[0, 0]",5.0,"I love this calculator,big numbers and calcu ...","12 2, 2013",A1RTVWTWZSIC94,"Hb ""Black Beauty"""


In [33]:
## Counting the number of rows 
len(Office_Products)

53258

In [86]:
## Setting target as the notebook
graphlab.canvas.set_target('ipynb')

In [87]:
## Checking the rating distribution. Most reviews have a 5 rating. 30,327 votes (56.944%)
Office_Products['overall'].show(view='Categorical') 

In [88]:
# building a word count vector for each review 
Office_Products['word_count'] = graphlab.text_analytics.count_words(Office_Products['reviewText'])

In [89]:
# Removing all stopwords
Office_Products['No_Stop_words'] = Office_Products['word_count'].dict_trim_by_keys(graphlab.text_analytics.stopwords(), exclude=True)

In [90]:
# Limiting the review data to only helpful reviews. The ‘helpful’ column contains values like ‘[x, y]’.
# The first value represents the number of helpful votes, the second represents overall votes. 
# If at least 60% consider the review helpful, we want those reviews. A binary 1 or 0 column will identify the review as helpful or not.
import numpy as np

Office_Products['helpful_count'] = Office_Products['helpful'].apply(lambda x: x[0])
Office_Products['overall_count'] = Office_Products['helpful'].apply(lambda x: x[1])
Office_Products['helpful_percent'] = Office_Products['helpful_count'] / Office_Products['overall_count']
Office_Products['helpful_review'] = Office_Products['helpful_percent'] >= .6
Office_Products = Office_Products[Office_Products['overall_count']!= 0 ]


In [91]:
Office_Products.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID
B00000JBLH,"[3, 4]",5.0,"I bought my first HP12Cin about 1984 or so, and ...","09 3, 2004",A32T2H8150OJLU
B00000JBLH,"[7, 9]",5.0,WHY THIS BELATED REVIEW?I feel very obliged to ...,"12 15, 2007",A3MAFS04ZABRGO
B00000JBLH,"[3, 3]",2.0,I have an HP 48GX thathas been kicking for ...,"01 1, 2011",A1F1A0QQP2XVH5
B00000JBLH,"[7, 8]",5.0,I've started doing morefinance stuff recently ...,"04 19, 2006",A49R5DBXXQDE5
B00000JBLH,"[10, 12]",5.0,"While I don't have anMBA, it's hard to bel ...","01 23, 2002",A2JFOHC9W629IE
B00000JBLH,"[3, 4]",5.0,I've had an HP 12C eversince they were first ...,"01 17, 2007",A38NELQT98S4H8
B00000JBLU,"[3, 3]",5.0,"This is a well-designed,simple calculator that ...","12 7, 2010",A25C2M3QF9G7OQ
B00000JBLU,"[2, 2]",5.0,I've had mine for 4 yearsnow and use it almost ...,"07 10, 2007",A30U2QQN2FFHE9
B00000JBLU,"[1, 2]",5.0,"If you need a calculatorthat prints, this is not ...","07 15, 2008",A2W0XCWOK3MW6F
B00000JBLU,"[6, 6]",5.0,"Texas Instruments makesan excellent, relatively ...","06 22, 2010",A2R6RA8FRBS608


In [92]:
# Building the sentiment classifier. Performing a random 80/20 split to select the training and testing data.
train_data, test_data = Office_Products.random_split(.8, seed=0)

In [93]:
sentiment_model_Logist = graphlab.logistic_classifier.create(train_data,
                                                            target = 'helpful_review',
                                                            features = ['No_Stop_words'],
                                                            validation_set = test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 16902

Number of classes           : 2

...

| 6         | 14       | 1.000000  | 2.339133     | 0.984795          | 0.664000            |

| 10        | 19       | 1.000000  | 3.720212     | 0.993196          | 0.687515            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [94]:
## evaluating the model
sentiment_model_Logist.evaluate(test_data, metric = 'roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+------+------+
 | threshold |      fpr       |      tpr       |  p   |  n   |
 +-----------+----------------+----------------+------+------+
 |    0.0    |      1.0       |      1.0       | 2938 | 1187 |
 |   1e-05   | 0.996630160067 | 0.999659632403 | 2938 | 1187 |
 |   2e-05   | 0.995787700084 | 0.999659632403 | 2938 | 1187 |
 |   3e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   4e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   5e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   6e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   7e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   8e-05   | 0.995787700084 | 0.998978897209 | 2938 | 1187 |
 |   9e-05   | 0.994945240101 | 0.998978897209 | 2938 | 1187 |
 +-----------+----------------+----------------+------+------+
 [100001 row

In [96]:
# True Positive = 1189 False Negative = 1718 Accuracy = 0.685 Precision = 0.423 false Positive = 1625 True Negative = 6077 recall = 0.409
sentiment_model_Logist.show(view = 'Evaluation')

In [97]:
## Trying a SVM model
sentiment_model_SVM = graphlab.svm_classifier.create(train_data,
                                                     target = 'helpful_review',
                                                     features = ['No_Stop_words'],
                                                     validation_set = test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

SVM:

--------------------------------------------------------

Number of examples          : 16902

Number of classes           : 2

...

| 6         | 14       | 1.000000  | 2.096120     | 0.976748          | 0.700848            |

| 10        | 18       | 1.000000  | 2.916167     | 0.990001          | 0.702303            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [98]:
## Evaluating the model
coefficients = sentiment_model_SVM['coefficients'] 

In [99]:
# Make predictions (as margins, or class)
predictions = sentiment_model_SVM.predict(Office_Products)    # Predicts 0/1
predictions = sentiment_model_SVM.predict(Office_Products, output_type='margin')

In [100]:
results = sentiment_model_SVM.evaluate(Office_Products)  

In [101]:
results

{'accuracy': 0.9335616112617111, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  245  |
 |      0       |        1        |  1152 |
 |      0       |        0        |  4843 |
 |      1       |        1        | 14787 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns], 'f1_score': 0.9548932872687352, 'precision': 0.9277244494635799, 'recall': 0.9837014369345396}

In [102]:
## Finding the ID of the most frequently reviewed item. 
Office_Products['asin'].show()   ## B0010T3QT2 count = 311 (58.4% of the reviews)

In [103]:
B0035FX5MC_Reviews = Office_Products[Office_Products['asin'] == 'B0035FX5MC']

In [104]:
len(B0035FX5MC_Reviews)

122

In [105]:
# Applying the SVM model
Office_Products['Predicted_Helpfulness_SVM'] = sentiment_model_SVM.predict(Office_Products, output_type = 'margin')

In [106]:
## Sorting reviews based on the predicted sentiment 
Office_Products = Office_Products.sort('Predicted_Helpfulness_SVM', ascending =False)

In [107]:
# Applying the Logistic Regression model
Office_Products['Predicted_Helpfulness_Logist'] = sentiment_model_Logist.predict(Office_Products, output_type = 'probability')

In [108]:
## Sorting reviews based on the predicted sentiment 
Office_Products = Office_Products.sort('Predicted_Helpfulness_Logist', ascending =False)

In [109]:
Office_Products.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID
B005D5M12M,"[78, 79]",5.0,I've owned an HP colorLaserJet for quite some ...,"08 23, 2012",A12B7ZMXFI6IXY
B001S9HWKS,"[4, 4]",5.0,"Excellent Investment.Easyto use.Intuitive, simple ...","06 18, 2011",A2V5UI37R7ESEV
B002JM1XRQ,"[21, 21]",5.0,This is quite possiblythe best printer that ...,"01 18, 2010",A38NELQT98S4H8
B0063B8B92,"[18, 20]",5.0,Our house is starting tolook something like a ...,"12 28, 2011",AUITG1DJ3QUGK
B004H3XKR6,"[32, 34]",5.0,INSTALLATION PROCESS:***Smooth and intuitive. I ...,"02 1, 2011",A2VWE5SGNDS8HW
B005IVL0RS,"[30, 33]",3.0,Update - 1/6/2012 - Arecent firmware release ...,"12 9, 2011",A38NELQT98S4H8
B001R4BTI0,"[43, 45]",4.0,"I have used an HP Scanjet4670 for years, and ...","03 6, 2011",A1GQGYROVZVW49
B002MCZIUG,"[11, 11]",4.0,Going against the oldadage of don't fix what ...,"11 28, 2009",A2BMZRO0H7TFCS
B0001EMLZW,"[106, 111]",5.0,Have been a stalwart HPfinancial calculator ...,"03 3, 2005",A33TZR3VQJMKIZ
B0064VZD06,"[131, 134]",5.0,The quality of the printsout of this printer is ...,"06 28, 2012",ASFP250F22MDZ


In [110]:
Office_Products[0]['reviewText']

'I\'ve owned an HP color LaserJet for quite some time now, but the only problem is that is streaks. Something is wrong with the fuser/toner cartridges and despite me getting a local printer/copier technician to look at it, he thinks it can\'t be fixed. Not wanting to spend any more money trying to get it repaired, I\'m just holding onto it until the toner runs out.But recently, my my wife is planning on going back to college. Back to college means writing papers, printing out stuff, and all that jazz and she needed a printer that didn\'t have a huge streak going down each sheet. On top of that, she wanted a printer that was capable of copying/scanning. Having procrastinated for a while now, I did notice that another Canon Pixma (PIXMA MG6220) was the Gold Box Deal of the Day. That printer retailed $199.99 but was on sale for $59.99. Instead of pulling the trigger on it, I decided to wait until she woke up. Unfortunately, that printer sold out lightning fast. When I checked back, some k

In [111]:
Office_Products[-1]['reviewText']

"UPDATE, 12/09/13: this machine is WAY OVERPRICED, now.  I got mine for maybe $400, will check and write an updated note in the comments.  It wasn't worth the amount I paid, much less the selling price now.UPDATE 12/09/13: I finally gave up and reinstalled my prior MFC 9700.  It scans well.  This 8480DN cannot scan directly from the machine, but ONLY from the computer.  However, at least the 8480DN has a power button, along with wire hookup in two ways, parallel and USB.  It also allows you to configure it from the computer, without having to use that HORRIBLE LCD menu.  But I gotta tell ya, I'll never buy another Brother printer again.  They didn't learn any lessons from the insanity of the MFC 9700's frustrating interface.UPDATE 3/22/13:  I upgraded one star for the printer, because there is a way to make it do searchable pdf via its PaperPort function.  But the scanner itself can't do that.  YOU can do tht, if you have Acrobat;  its OCR function will also convert any Google doc into

asin,helpful,overall,reviewText,reviewTime,reviewerID
B001V9LQH0,"[44, 48]",4.0,UPDATE 12/06/13: I'mreally surprised that ...,"12 2, 2011",A2LEIANN1UZTHP
B0095F5BCS,"[254, 265]",5.0,&#2951; Fuzzy Wuzzy's Summary:&#1150;&#1150;&# ...,"06 6, 2013",A24HWYHR28JJ7A
B003VQR1UC,"[1382, 1417]",5.0,"SUMMARYWith a fewcaveats, this is a great ...","10 8, 2010",AI0BCEWRE04G0
B003YL412U,"[469, 479]",4.0,!!!WARNING!!! B200ERROR:I wrote this re ...,"09 26, 2010",A2VW4FYZILSXF2
B003VQR1TS,"[612, 624]",5.0,"SUMMARYWith a fewcaveats, this is a great ...","10 3, 2010",AI0BCEWRE04G0
B001B19XPA,"[121, 131]",4.0,&#2951; Fuzzy Wuzzy's Summary:&#1150;&#1150;&# ...,"09 25, 2008",A24HWYHR28JJ7A
B00EHDZMAY,"[561, 567]",4.0,Canon PIXMA MG7120Black/White/Red/Brown ...,"12 13, 2013",A3D7BGK0P5QY0X
B00AGV7T5W,"[832, 849]",4.0,Canon PIXMA MG5420Wireless Color Photo ...,"03 11, 2013",A3D7BGK0P5QY0X
B00HM0IV4S,"[2, 2]",4.0,Pros:Excellent textqualityGood picture ...,"04 27, 2014",A2VW4FYZILSXF2
B00AGV7TOS,"[800, 812]",4.0,Canon PIXMA MG6320 Black(or White) Wireless C ...,"01 28, 2013",A3D7BGK0P5QY0X


'UPDATE 12/06/13:  I\'m really surprised that the price here has more than doubled since I bought it.  There is one really important thing to remember, too: the installation program which might be on your disk, might not be the right one.  I just spent six hours trying to install this thing on another computer, so to spare you the same hassle: hit the \'CONTACT US\' button on the CD when it loads.  Then search on \'Scansnap S1500 downloads\' and see if there\'s a later software program.  Mine was 5.0, and it didn\'t work (probably because Windows Updates trash everything, so they had to rewrite the program).  But when I just now downloaded the 5.1 program (which hahahaha runs in a DOS window), it works just fine.I still love my Scansnap.  Couldn\'t be in business without it.  Still haven\'t done the things listed in the review below which I said I would test.  Been spending the last year on learning Win7, 8, and WINDOWS GLITCHES eat my life.  Sorry...UPDATE re Linux, 9/8/12:  Scansnap 

'&#2951; Fuzzy Wuzzy\'s Summary:&#1150;&#1150;&#1150;&#1150;&#1150; Highly recommended with warm fuzzies!&#1411; Positives:&#1411; This printer can output superb gallery-quality prints with exceptional detail and resolution, while being significantly less expensive than the higher-end Pro-10 and Pro-1 printer models.&#1411; This Pro-100 prints better blacks and grays on monochrome prints than the older Pro9000 printer that it replaces.&#1411; The new wireless connectivity options of using 802.11b/g/n Wi-Fi or Apple\'s AirPrint, along with a wired Ethernet option, provide flexibility of shared usage and printer placement by not needing to be physically connected to a USB cable.&#1411; The dye-based inks used in this Pro-100 can retain their colors for 100+ years.&#4335; Negatives:&#4335; While black-and-white prints are better than the previous Pro9000 printer, monochrome output is still inferior to the pricier Pro-10 and Pro-1 printers.If you have never used a prosumer-level dedicated 

"UPDATE, 12/09/13: this machine is WAY OVERPRICED, now.  I got mine for maybe $400, will check and write an updated note in the comments.  It wasn't worth the amount I paid, much less the selling price now.UPDATE 12/09/13: I finally gave up and reinstalled my prior MFC 9700.  It scans well.  This 8480DN cannot scan directly from the machine, but ONLY from the computer.  However, at least the 8480DN has a power button, along with wire hookup in two ways, parallel and USB.  It also allows you to configure it from the computer, without having to use that HORRIBLE LCD menu.  But I gotta tell ya, I'll never buy another Brother printer again.  They didn't learn any lessons from the insanity of the MFC 9700's frustrating interface.UPDATE 3/22/13:  I upgraded one star for the printer, because there is a way to make it do searchable pdf via its PaperPort function.  But the scanner itself can't do that.  YOU can do tht, if you have Acrobat;  its OCR function will also convert any Google doc into

'I\'ve owned an HP color LaserJet for quite some time now, but the only problem is that is streaks. Something is wrong with the fuser/toner cartridges and despite me getting a local printer/copier technician to look at it, he thinks it can\'t be fixed. Not wanting to spend any more money trying to get it repaired, I\'m just holding onto it until the toner runs out.But recently, my my wife is planning on going back to college. Back to college means writing papers, printing out stuff, and all that jazz and she needed a printer that didn\'t have a huge streak going down each sheet. On top of that, she wanted a printer that was capable of copying/scanning. Having procrastinated for a while now, I did notice that another Canon Pixma (PIXMA MG6220) was the Gold Box Deal of the Day. That printer retailed $199.99 but was on sale for $59.99. Instead of pulling the trigger on it, I decided to wait until she woke up. Unfortunately, that printer sold out lightning fast. When I checked back, some k

asin,helpful,overall,reviewText,reviewTime,reviewerID
B005D5M12M,"[78, 79]",5.0,I've owned an HP colorLaserJet for quite some ...,"08 23, 2012",A12B7ZMXFI6IXY
B001S9HWKS,"[4, 4]",5.0,"Excellent Investment.Easyto use.Intuitive, simple ...","06 18, 2011",A2V5UI37R7ESEV
B002JM1XRQ,"[21, 21]",5.0,This is quite possiblythe best printer that ...,"01 18, 2010",A38NELQT98S4H8
B0063B8B92,"[18, 20]",5.0,Our house is starting tolook something like a ...,"12 28, 2011",AUITG1DJ3QUGK
B004H3XKR6,"[32, 34]",5.0,INSTALLATION PROCESS:***Smooth and intuitive. I ...,"02 1, 2011",A2VWE5SGNDS8HW
B005IVL0RS,"[30, 33]",3.0,Update - 1/6/2012 - Arecent firmware release ...,"12 9, 2011",A38NELQT98S4H8
B001R4BTI0,"[43, 45]",4.0,"I have used an HP Scanjet4670 for years, and ...","03 6, 2011",A1GQGYROVZVW49
B002MCZIUG,"[11, 11]",4.0,Going against the oldadage of don't fix what ...,"11 28, 2009",A2BMZRO0H7TFCS
B0001EMLZW,"[106, 111]",5.0,Have been a stalwart HPfinancial calculator ...,"03 3, 2005",A33TZR3VQJMKIZ
B0064VZD06,"[131, 134]",5.0,The quality of the printsout of this printer is ...,"06 28, 2012",ASFP250F22MDZ


asin,helpful,overall,reviewText,reviewTime,reviewerID
B009NSGO3M,"[0, 2]",4.0,****The 3M Easy-AdjustDual Monitor Arm ...,"10 11, 2013",A2AWVROFGSZU4E
B0074ZN27C,"[3, 6]",4.0,First up the packaging.The box seems a little ...,"03 21, 2012",A25QJBK33C4O0R
B004HY9IJE,"[0, 1]",3.0,With 61 out of 92 peoplerating this a 4 or 5 out ...,"06 30, 2014",A1SYLII0808HD6
B001GXM68Y,"[2, 7]",5.0,This CASIO HS-8VAcalculator is presently ...,"07 10, 2011",A2A1XYSB692L6J
B000OOYECC,"[1, 2]",4.0,I write for a living. Iuse this stand with a ...,"11 6, 2013",A3775OP5VTX5ON
B004H3XKR6,"[0, 1]",4.0,SetupSetting up was easyenough I didn't even ...,"03 17, 2011",AR3EVUQF0AC7R
B001B0BE6M,"[1, 10]",3.0,You place one or moreobjects inside the clear ...,"06 16, 2011",A2AWVROFGSZU4E
B00007LVCN,"[3, 8]",4.0,The Avery 8395 labels --generically called name ...,"04 10, 2011",A2AWVROFGSZU4E
B005HFJFK4,"[4, 19]",2.0,I have owned and/orreviewed over 5 diffe ...,"03 21, 2012",APDPA11IZPYLN
B001YTK3XK,"[0, 1]",2.0,"UPDATE, 12/09/13: thismachine is WAY ...","09 11, 2012",A2LEIANN1UZTHP
